In [ ]:
""" LangSmith 트레이싱 - 세션 시작/재시작 후 맨 먼저 실행 """
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = "lsv2_*"  # smith.langchain.com → Settings → API Keys
os.environ["LANGCHAIN_PROJECT"] = "TEST"

### Long-term memory 구현
### method 별 args 는 docs에서 파악이 필수적

In [ ]:
from dataclasses import dataclass
from langgraph.store.memory import InMemoryStore
from langchain.agents.middleware import wrap_model_call
from langchain.messages import HumanMessage, SystemMessage

# 실행 컨텍스트 정의
@dataclass  # 유저정보.
class Context:
    user_id: str  # 유저 아이디 (폴더명)
    app_name: str # 유저 데이터 (파일명)

# Store 초기화 - 메모리 담아 둘 Store.
store = InMemoryStore()

from typing import TypedDict

# LLM이 추출해야 할 정보의 구조를 정의
class UserInfo(TypedDict):
    personal_info : str
    preference : str

In [ ]:
from langchain_core.runnables import RunnableConfig
from langchain.tools import tool, ToolRuntime

# Tool 정의: 사용자의 정보를 조회하는 tool
@tool
def get_user_info(runtime) -> str:
    """
    현재 사용자의 정보 조회 (시스템 내부용 도구)
    """
    ctx = runtime.context
    if ctx is None:
        return "오류: 실행 컨텍스트가 없습니다. invoke 시 context=Context(...) 를 전달하세요."
    user_id = ctx.user_id
    app = ctx.app_name

    # 해당 네임스페이스의 모든 메모리 검색
    memories = runtime.store.search((user_id, app))   # store에는 중복이 없도록 set = () 로 정의됨

    if not memories:
        return "기록된 정보 없음"

    results = []
    for item in memories:
        data = item.value or {}
        for k, v in data.items():
            if v is not None and str(v).strip():
                results.append(f"- {k}: {v}")

    return "\n".join(results) if results else "기록된 정보 없음"

In [4]:
import uuid

@tool
def save_user_info(user_info, runtime):
    """
    사용자의 정보를 저장하거나 업데이트
    """
    ctx = runtime.context
    if ctx is None:
        return "오류: 실행 컨텍스트가 없습니다. invoke 시 context=Context(...) 를 전달하세요."
    user_id = ctx.user_id
    app = ctx.app_name
    store = runtime.store

    # 2. Store에 데이터 저장 (put(namespace, key, value))
    memory_key = str(uuid.uuid4())
    store.put((user_id, app), memory_key, user_info)
    # 저장 직후 get으로 확인해 반환 (저장 검증 + 디버깅)
    saved = store.get((user_id, app), memory_key)
    content = saved.value if saved else user_info
    return f"정보가 안전하게 저장되었습니다. (ID: {memory_key})\n저장된 내용: {content}"

In [ ]:
from langchain.chat_models import init_chat_model
from langchain.agents import create_agent

model = init_chat_model(
    model="llama3.2:3b", 
    model_provider="ollama",
    temperature=0.8,
)

agent = create_agent(
    model=model,
    tools=[get_user_info, save_user_info],
    store=store, # 에이전트에 store 연결
    context_schema=Context # 조회할 유저 정보로 사전에 정의해두었음.
)

In [9]:
# context는 invoke의 context= 인자로 넘겨야 툴 runtime.context에 전달됨 (config만으로는 None)
from langchain_core.runnables import RunnableConfig

_ctx = Context(user_id="user_001", app_name="personal_assistant")
response = agent.invoke(
    {"messages": [{"role": "user", "content": "My name is now 'Jtoh'. I prefer tea over coffee."}]},
    config=RunnableConfig(configurable={"context": _ctx}),
    context=_ctx
)

In [10]:
response["messages"][-1].content

"Hello Jtoh!\n\nI've taken note of your preference for tea over coffee. I'll make sure to suggest tea options whenever we chat or provide recommendations.\n\nHow can I assist you today? Would you like some tea pairing suggestions or perhaps a recommendation on a new tea brand to try?"

In [13]:
from langchain_core.runnables import RunnableConfig

_ctx = Context(user_id="user_001", app_name="personal_assistant")
response = agent.invoke(
    {"messages": [{"role": "user", "content": "Tell me what you know about me"}]},
    config=RunnableConfig(configurable={"context": _ctx}),
    context=_ctx
)

In [14]:
response["messages"][-1].content

"Based on the user info, it appears that the user's name is Jtoh, and their preferred beverage is tea. Is there anything else I can help you with?"

만약 user_002 로 하면 ? (다른 정보로 대입)

In [15]:
from langchain_core.runnables import RunnableConfig

_ctx = Context(user_id="user_002", app_name="personal_assistant")
response = agent.invoke(
    {"messages": [{"role": "user", "content": "Tell me what you know about me"}]},
    config=RunnableConfig(configurable={"context": _ctx}),
    context=_ctx
)
response["messages"][-1].content

"I couldn't find any information about the user, as there was no input provided when I first started our conversation. However, I can try to gather some information based on our current conversation.\n\nIt seems that you're a user who interacted with me through a text-based interface, and you're curious about what I know about you. Unfortunately, my knowledge is limited to the information provided during our conversation, and I don't have any prior knowledge or records about individual users.\n\nIf you'd like to share some information about yourself, I'm happy to learn more and try to provide a personalized response."

In [16]:
from langchain_core.runnables import RunnableConfig

_ctx = Context(user_id="user_001", app_name="favorite")
response = agent.invoke(
    {"messages": [{"role": "user", "content": "Tell me what you know about me"}]},
    config=RunnableConfig(configurable={"context": _ctx}),
    context=_ctx
)
response["messages"][-1].content

"I couldn't find any information about the user. It's as if I'm starting from a blank slate.\n\nHowever, I can start fresh and try to gather some information about you through our conversation. What would you like to talk about? How can I assist you today?"

## DB에서 조회하고 땡겨온 걸 넣으면 자연스러워 지겠다.

# RAG

### pip install  특정 로더 ( PDFPlumberLoader ) / 임베딩 모델 (llama qwen3-embedding)/ langchain-text-splitters / 등 자유롭게 설치 및 활용

### Docs 활용 필요
https://docs.langchain.com/oss/python/integrations/text_embedding
https://docs.ollama.com/capabilities/embeddings

LOAD - SPLIT

In [17]:
from langchain_community.document_loaders import PDFPlumberLoader

file_path = "ARK_INNOVATION_ETF_ARKK_HOLDINGS.pdf"
loader = PDFPlumberLoader(file_path)

In [18]:
docs = loader.load()

In [19]:
docs

[Document(metadata={'source': 'ARK_INNOVATION_ETF_ARKK_HOLDINGS.pdf', 'file_path': 'ARK_INNOVATION_ETF_ARKK_HOLDINGS.pdf', 'page': 0, 'total_pages': 3, 'Keywords': 'HD:2025-11-26', 'CreationDate': "D:20251126070539+00'00'", 'ModDate': "D:20251126070539+00'00'"}, page_content='Holdings Data - ARKK\nAs of 11/26/2025\nARKK\nARK Innovation ETF\nCompany Ticker CUSIP Shares Market Value ($) Weight (%)\n1 TESLA INC TSLA 88160R101 2,204,438 $924,541,297.20 12.26%\n2 TEMPUS AI INC TEM 88023B103 5,465,331 $419,956,034.04 5.57%\n3 ROKU INC ROKU 77543R102 4,347,025 $412,445,732.00 5.47%\n4 CRISPR THERAPEUTICS AG CRSP H17182108 7,625,349 $408,489,945.93 5.42%\n5 COINBASE GLOBAL INC -CLASS A COIN 19260Q107 1,542,644 $392,016,693.28 5.20%\n6 SHOPIFY INC - CLASS A SHOP 82509L107 2,415,270 $380,091,039.90 5.04%\n7 ROBINHOOD MARKETS INC - A HOOD 770700102 2,751,983 $318,046,675.31 4.22%\n8 ROBLOX CORP -CLASS A RBLX 771049103 3,142,157 $286,156,237.99 3.79%\n9 PALANTIR TECHNOLOGIES INC-A PLTR 69608A108 1

In [20]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 300,
    chunk_overlap = 10
)

recursive_docs = text_splitter.split_documents(docs)
recursive_docs

[Document(metadata={'source': 'ARK_INNOVATION_ETF_ARKK_HOLDINGS.pdf', 'file_path': 'ARK_INNOVATION_ETF_ARKK_HOLDINGS.pdf', 'page': 0, 'total_pages': 3, 'Keywords': 'HD:2025-11-26', 'CreationDate': "D:20251126070539+00'00'", 'ModDate': "D:20251126070539+00'00'"}, page_content='Holdings Data - ARKK\nAs of 11/26/2025\nARKK\nARK Innovation ETF\nCompany Ticker CUSIP Shares Market Value ($) Weight (%)\n1 TESLA INC TSLA 88160R101 2,204,438 $924,541,297.20 12.26%\n2 TEMPUS AI INC TEM 88023B103 5,465,331 $419,956,034.04 5.57%\n3 ROKU INC ROKU 77543R102 4,347,025 $412,445,732.00 5.47%'),
 Document(metadata={'source': 'ARK_INNOVATION_ETF_ARKK_HOLDINGS.pdf', 'file_path': 'ARK_INNOVATION_ETF_ARKK_HOLDINGS.pdf', 'page': 0, 'total_pages': 3, 'Keywords': 'HD:2025-11-26', 'CreationDate': "D:20251126070539+00'00'", 'ModDate': "D:20251126070539+00'00'"}, page_content='4 CRISPR THERAPEUTICS AG CRSP H17182108 7,625,349 $408,489,945.93 5.42%\n5 COINBASE GLOBAL INC -CLASS A COIN 19260Q107 1,542,644 $392,016,

In [21]:
len(recursive_docs)

25

In [22]:
recursive_docs[0].page_content

'Holdings Data - ARKK\nAs of 11/26/2025\nARKK\nARK Innovation ETF\nCompany Ticker CUSIP Shares Market Value ($) Weight (%)\n1 TESLA INC TSLA 88160R101 2,204,438 $924,541,297.20 12.26%\n2 TEMPUS AI INC TEM 88023B103 5,465,331 $419,956,034.04 5.57%\n3 ROKU INC ROKU 77543R102 4,347,025 $412,445,732.00 5.47%'

In [ ]:
len(recursive_docs[0].page_content)  # 최대 300개 token 아래로 chunk_size 정의함

297

Embedding

ollama pull qwen3-embedding (임베딩모델)

In [24]:
from langchain_ollama import OllamaEmbeddings
from langchain_classic.storage import LocalFileStore  # (DB대용)
from langchain_classic.embeddings import CacheBackedEmbeddings # 캐시로 임베딩 계속 안하도록 처리 

underlying_embeddings = OllamaEmbeddings(
    model="qwen3-embedding", 
)

store = LocalFileStore("./cache/")

cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    underlying_embeddings,
    store,
    namespace=underlying_embeddings.model
    )

c:\Users\jtoh\anaconda3\envs\langchain\Lib\site-packages\langchain_classic\embeddings\cache.py:58: UserWarning: Using default key encoder: SHA-1 is *not* collision-resistant. While acceptable for most cache scenarios, a motivated attacker can craft two different payloads that map to the same cache key. If that risk matters in your environment, supply a stronger encoder (e.g. SHA-256 or BLAKE2) via the `key_encoder` argument. If you change the key encoder, consider also creating a new cache, to avoid (the potential for) collisions with existing keys.
  _warn_about_sha1_encoder()


In [26]:
from langchain_core.vectorstores import InMemoryVectorStore

vectorstore = InMemoryVectorStore.from_documents(
    recursive_docs,
    cached_embedder
)

In [27]:
query = "what is TESLA INC or TSLA ratio"
results = vectorstore.similarity_search(query)
results

[Document(id='f0ecb251-fdc6-4501-bcdc-3957d1270db2', metadata={'source': 'ARK_INNOVATION_ETF_ARKK_HOLDINGS.pdf', 'file_path': 'ARK_INNOVATION_ETF_ARKK_HOLDINGS.pdf', 'page': 0, 'total_pages': 3, 'Keywords': 'HD:2025-11-26', 'CreationDate': "D:20251126070539+00'00'", 'ModDate': "D:20251126070539+00'00'"}, page_content='Holdings Data - ARKK\nAs of 11/26/2025\nARKK\nARK Innovation ETF\nCompany Ticker CUSIP Shares Market Value ($) Weight (%)\n1 TESLA INC TSLA 88160R101 2,204,438 $924,541,297.20 12.26%\n2 TEMPUS AI INC TEM 88023B103 5,465,331 $419,956,034.04 5.57%\n3 ROKU INC ROKU 77543R102 4,347,025 $412,445,732.00 5.47%'),
 Document(id='02ddf416-2abb-4190-86d2-d45672f4a6dc', metadata={'source': 'ARK_INNOVATION_ETF_ARKK_HOLDINGS.pdf', 'file_path': 'ARK_INNOVATION_ETF_ARKK_HOLDINGS.pdf', 'page': 0, 'total_pages': 3, 'Keywords': 'HD:2025-11-26', 'CreationDate': "D:20251126070539+00'00'", 'ModDate': "D:20251126070539+00'00'"}, page_content='20 AMAZON.COM INC AMZN 023135106 568,449 $130,555,68

In [32]:
from pprint import pprint
pprint(results[0].page_content)

('Holdings Data - ARKK\n'
 'As of 11/26/2025\n'
 'ARKK\n'
 'ARK Innovation ETF\n'
 'Company Ticker CUSIP Shares Market Value ($) Weight (%)\n'
 '1 TESLA INC TSLA 88160R101 2,204,438 $924,541,297.20 12.26%\n'
 '2 TEMPUS AI INC TEM 88023B103 5,465,331 $419,956,034.04 5.57%\n'
 '3 ROKU INC ROKU 77543R102 4,347,025 $412,445,732.00 5.47%')


In [33]:
# 추후 시간이 나면 업데이트 예정..